# throughput 결과 비교

In [2]:
import pandas as pd
import os
import numpy as np
from scipy import stats
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
import math
import numpy as np
from numpy.random import randn
from statsmodels.stats.weightstats import ztest

In [171]:
# 경로 내 파일 확인
os.listdir('./result_data')

['gp_rdb_external_4_v1.csv',
 'result_data_internal.csv',
 'gp_aof_external_4_v21.csv',
 'gp_rdb_external_4_v2.csv',
 '.DS_Store',
 'gp_rdb_internal_4_v2.csv',
 'gp_rdb_internal_4_v1.csv',
 'gp_aof_external_4_v2.csv',
 'gp_T_rdb_external_4.csv',
 'gp_aof_external_4_v1.csv',
 'gp_rdb_external_4_v21.csv',
 'gp_aof_internal_4_v1.csv',
 'gp_aof_internal_4_v2.csv',
 'result_data_external.csv',
 'gp_rdb_external_4_test.csv',
 'gp_rdb_external_4_v11.csv',
 'check_rdb_internal.csv',
 '.ipynb_checkpoints',
 'check.csv',
 'check_rdb_external.csv',
 'default_rdb_external_4.csv',
 'default_aof_external_4.csv',
 'gp_aof_external_4_v11.csv']

In [218]:
check_rdb_internal = pd.read_csv('./result_data/check_rdb_internal.csv', sep=',')
check_rdb_external = pd.read_csv('./result_data/check_rdb_external.csv', sep=',')
check_rdb_external_th = check_rdb_external['Gets_KB/sec']

In [219]:
check_rdb_internal['config_file'][0]

'/home/swredis/redis-sample-generation/configfiletest/rdb1/config_rdb1_rf_T.conf'

In [201]:
# configfile model inetrnal에서 인덱스 추출

def get_config_model_idx(internalcsv_df, config_name_list):
    config_name_data = list(internalcsv_df['config_file'])
    
    # 딕셔너리 미리 선언
    config_idx_dict = {}
    
    for config_name in config_name_list:
        print(config_name)
        # 리스트 선언
        config_idx_dict[config_name] = []
        
        for idx, config in enumerate(config_name_data):
            if config_name == config[58:]:
                config_idx_dict[config_name].append(idx)
                
    return config_idx_dict
                
        
    


In [213]:
def get_mean_th(config_name):
    # internal.csv에서 idx 추출
    config_name_list = [config_name+'_T.conf', config_name+'.conf']
    idx_dict = get_config_model_idx(check_rdb_internal, config_name_list)
    
    # 인덱스 출력
    print(idx_dict)
    
    # total bo 출력
    print('total = ', check_rdb_external_th.iloc[idx_dict[config_name+'_T.conf']].mean())
    # grouped bo 출력
    print('grouped = ',check_rdb_external_th.iloc[idx_dict[config_name+'.conf']].mean())

In [220]:
get_mean_th('config_rdb1_rf')

config_rdb1_rf_T.conf
config_rdb1_rf.conf
{'config_rdb1_rf_T.conf': [0, 2, 4, 6, 8, 10, 12, 14, 16, 18], 'config_rdb1_rf.conf': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19]}
total =  57265.045
grouped =  60467.189


In [204]:
# internal.csv에서 idx 추출
config_name_list = ['config_rdb1_xgb_T.conf', 'config_rdb1_xgb.conf']
idx_dict = get_config_model_idx(check_rdb_internal, config_name_list)

config_rdb1_xgb_T.conf
config_rdb1_xgb.conf


In [184]:
idx_dict

{'config_rdb1_xgb_T.conf': [0, 2, 4, 6, 8, 10, 12, 14, 16, 18],
 'config_rdb1_xgb.conf': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19]}

In [187]:
check_rdb_external_th.iloc[idx_dict['config_rdb1_xgb_T.conf']].mean()

57555.856999999996

In [186]:
check_rdb_external_th.iloc[idx_dict['config_rdb1_xgb.conf']].mean()

55839.91900000001

In [161]:
# ttest
x1 = check_rdb_external_th.iloc[idx_dict['config_rdb1_lightgbm.conf']] 
x2 = check_rdb_external_th.iloc[idx_dict['config_rdb1_lightgbm_T.conf']]
sp.stats.ttest_ind(x1, x2, equal_var=True, alternative='greater')

Ttest_indResult(statistic=1.0089779801972019, pvalue=0.16317851167779113)

In [162]:
# 정규성 검정
x1 = check_rdb_external_th.iloc[idx_dict['config_rdb1_lightgbm.conf']]
print(stats.shapiro(x1))

x1 = check_rdb_external_th.iloc[idx_dict['config_rdb1_lightgbm_T.conf']]
print(stats.shapiro(x1))

ShapiroResult(statistic=0.8984648585319519, pvalue=0.21073801815509796)
ShapiroResult(statistic=0.8973281383514404, pvalue=0.20473258197307587)


In [163]:
# 순위합 검정
x1 =  check_rdb_external_th.iloc[idx_dict['config_rdb1_lightgbm.conf']] 
x2 =  check_rdb_external_th.iloc[idx_dict['config_rdb1_lightgbm_T.conf']] 
stats.ranksums(x1, x2, alternative='greater')

RanksumsResult(statistic=1.209486313629527, pvalue=0.11323803302174312)

---

## 결과 비교

### RDB 1:1
* lightgbm : ligthgbm_T = 60218.06100000001 : 57734.051
* xgb : xgb_T =  57253.51400000001: 56484.08600000001
* dt : dt_T = 60401.24899999999 : 59377.212
* gbm : gbm_T =  60629.213: 56246.085
* rf : rf_T =  60467.189: 57265.045
* ridge : ridge_T = 58515.275 : 57963.73299999999
* lasso : lasso_T = 60537.905999999995 : 54362.905999999995
* lr : lr_T =  60766.30699999999: 59428.975

60218,
57254,
60401,
60629,
60467,
58515,
60538,
60766,


57734,
56484,
59377,
56246,
57265,
57964,
54363,
59429,

### RDB 1:0
* lightgbm : ligthgbm_T = 61104.594 : 59341.672999999995
* xgb : xgb_T =  59431.960999999996: 57047.482
* dt : dt_T =  60647.54800000001: 58754.46799999999
* gbm : gbm_T =  59806.98300000001: 55797.536 
* rf : rf_T =  58704.784: 56872.00600000001
* ridge : ridge_T = 58830.56799999999:56211.77099999999
* lasso : lasso_T =  56719.488: 56055.80500000001 
* lr : lr_T =  60036.26399999999: 55539.239 


61105,
59432,
60648,
59807,
58705,
58831,
56719,
60036

59342
57047
58754
55798
56872
56212
56056
55539

### AOF 1:1
* lightgbm : ligthgbm_T =  47598.895: 46798.364
* xgb : xgb_T =  46737.944: 45804.748999999996 
* dt : dt_T =  48180.856: 46236.905000000006
* gbm : gbm_T = 47223.39200000001 :  46215.36
* rf : rf_T = 44559.83: 43553.254
* ridge : ridge_T = 48138.235: 45653.562
* lasso : lasso_T = 45938.11899999999 : 42203.963
* lr : lr_T =  47609.629: 48321.268


47599,
46738,
48181,
47223,
44560,
48138,
45938,
48321


46798,
45805,
46237,
46215,
43553,
45654,
42204,
47610


### AOF 1:0
* lightgbm : ligthgbm_T = 44520.18200000001 : 36878.71000000001
* xgb : xgb_T =  44380.258: 44004.786
* dt : dt_T = 47973.918000000005: 44776.588
* gbm : gbm_T =  45391.187000000005: 38095.614
* rf : rf_T = 46795.33500000001 : 43649.324
* ridge : ridge_T = 46167.246999999996: 43128.362
* lasso : lasso_T = 45073.066 : 42195.172999999995 
* lr : lr_T =  : 46739.427 : 44658.479999999996

44520,  # lightgbm
45988,  # xgb
47974,
45391,  # gbm
46795,
46167,  # ridge
45073,
46739


36879,  # lightgbm
44005,  # xgb
44777,
38096,  # gbm
43649,
43128,  # ridge
42195,
44658



In [8]:
rdb10_external = pd.read_csv('./result_data/result_rdb10_external.csv', sep=',')
rdb10_external['Gets_KB/sec']

0      66668.67
1      59460.81
2      61186.59
3      59748.04
4      62342.58
         ...   
495    48687.51
496    43818.14
497    52924.89
498    51332.20
499    48205.66
Name: Gets_KB/sec, Length: 500, dtype: float64